In [10]:
"""
python model_run.py VCN_c09 --type Bushy --modeltype II --protocol runANSingles -H --model XM13nacncoop  -r 5 --sgcmodel cochlea -S MS  -f 16000. --soma-autoinflate --dendrite-autoinflate --noparallel
# python model_run.py VCN_c09 --type Bushy --modeltype II --protocol runANSingles -H --model XM13nacncoop --inputpattern VCN_c10 -r 5 --sgcmodel cochlea -S MS  -f 16000. 
"""
%matplotlib qt
import matplotlib
matplotlib.use('Qt5Agg')
import pickle
import numpy as np
from cnmodel.util import sound
import matplotlib.pyplot as mpl
from pathlib import Path
import pylibrary.plotting.plothelpers as PH
from vcnmodel import cell_config

freq = 16000.
db = 30
SR = 'MS'
nreps = 1

cconfig = cell_config.CellConfig()
dendscf = {'VCN_c09': 1.236, 'VCN_c11': 0.980, 'VCN_c17': 1.119, 'VCN_c18': 1.162}
def get_sims(cell, inp):
    
    scf = cconfig.get_soma_ratio(cell)
    # fnb = 'VCN_Cells/VCN_c09/Simulations/AN/AN_Result_VCN_c09_inp=self'
    # fnb + '_XM13nacncoop_II_soma=1.489_dend=1.236_Syn000_multisite_001_tonepip_030dB_16000.0_HS.p'
    basedir = '/Users/pbmanis/Desktop/Python/VCNModel'
    fnb = f'VCN_Cells/{cell:s}/Simulations/AN/AN_Result_{cell:s}_inp={inp:s}'  # or input = 'VCN_c02' etc
    fnb += f'_XM13_nacncoop_II_soma={scf:.3f}_dend={dendscf[cell]:.3f}_ASA={scf:.3f}_Syn000_multisite_001_tonepip_{db:03d}dB_{freq:.1f}_{SR:2s}.p'
    print('fnb: ', fnb)
    fnsyn = Path(basedir, fnb[:fnb.find('_Syn000') + 4])
    fns = fnsyn.parent.glob(str(fnsyn.name)+'*.p')
    # print('fns: ', list(fns))
    fnsl0 = sorted(list(fns))
    fnsl = []
    for i in range(len(fnsl0)):
        if str(fnsl0[i]).find(f'_{nreps:03d}_tonepip') > 0 and str(fnsl0[i]).find(f'_{freq:.1f}') > 0:
            fnsl.append(fnsl0[i])
    nax = len(fnsl)
    for f in fnsl:
        print('name: ', str(f.name))
    return fnsl

def get_one(fn):
    d = pickle.load(open(fn, 'rb'))
#     print(d.keys())
#     print('NTrials: ', len(d['trials']))
#     print(d['Params'].keys())
    return d


           Cell ID               ASA  nsyn(calculated)             delay           SRgroup            delay2        axonlength     branch length           syntype      postlocation


In [6]:
def make_sound(stimInfo):
    if isinstance(stimInfo['pip_start'], list):
        pips = stimInfo['pip_start']
    else:
        pips = [stimInfo['pip_start']]
    if stimInfo['soundtype'] == 'tonepip':
        stim = sound.TonePip(rate=stimInfo['Fs'], duration=stimInfo['run_duration'],
                          f0=stimInfo['F0'], dbspl=stimInfo['dB'],
                          ramp_duration=stimInfo['RF'], pip_duration=stimInfo['pip_duration'],
                          pip_start=pips)
    elif stimInfo['soundtype'] == 'SAM':
        stim = sound.SAMTone(rate=stimInfo['Fs'], duration=stimInfo['run_duration'],
                          f0=stimInfo['F0'], dbspl=stimInfo['dB'],
                          ramp_duration=stimInfo['RF'], 
                          fmod=stimInfo['fmod'], dmod=stimInfo['dmod'],
                          pip_duration=stimInfo['pip_duration'],
                          pip_start=pips)
    else:
        raise ValueError('StimInfo sound type %s not implemented' % stimInfo['soundtype'])

    stimWaveform = stim.generate()
    stimTimebase = stim.time
    return(stimTimebase, stimWaveform)

In [7]:
def plot_sim(cell, inp):
    if inp != 'self':
        r, ct = cconfig.makeDict(inp)
    else:
        r, ct = cconfig.makeDict(cell)

    nax = len(r)
    
    P = PH.regular_grid(nax+1, 1, order='columnsfirst', figsize=(6., 10.), showgrid=False,
                    verticalspacing=0.01, horizontalspacing=0.1,
                    margins={'leftmargin': 0.07, 'rightmargin': 0.05, 'topmargin': 0.08, 'bottommargin': 0.1},
                    labelposition=(-0.15, 1.05), parent_figure=None, panel_labels=None)
    # f.figsize=((6,10))
    P.figure_handle.suptitle(f'{cell:s} with {inp:s} inputs')


    for i in range(nax):
        if i < nax:
            d = get_one(fnsl[i])
            if d is None:
                continue

            for j, t in enumerate(d['trials']['somaVoltage'].keys()):
                P.axarr[i, 0].plot(d['trials']['time'], d['trials']['somaVoltage'][j], linewidth=0.5)
                P.axarr[i, 0].set_ylim([-70., 0.])
                PH.nice_plot(P.axarr[i, 0])
                if i == nax-1:
                    PH.calbar(P.axarr[i, 0], [0., -40., 25, 10], unitNames={'x': 'ms', 'y': 'mV'})
                else:
                    PH.noaxes(P.axarr[i, 0])
            P.axarr[i, 0].text(x=0.9, y=0.5, s=f"{r[i]['asa']:.1f}", transform=P.axarr[i, 0].transAxes)
    stb, snd = make_sound(d['trials']['stimInfo'])
    P.axarr[nax, 0].plot(stb, snd, 'b', linewidth=0.5)
    PH.noaxes(P.axarr[nax, 0])

    #mpl.show()
    mpl.show()                    

In [8]:
inps = ['VCN_c02', 'VCN_c05', 'VCN_c10', 'VCN_c13'] # ['self']*4 # 'VCN_c13'
cells = ['VCN_c09']*4 #,['VCN_c09'm 'VCN_c11', 'VCN_c17', 'VCN_c18']
print(inps)
for i, cell in enumerate(cells):
    print(cell, inps[i])
    fnsl = get_sims(cell, inps[i])
    plot_sim(cell, inps[i])


['VCN_c02', 'VCN_c05', 'VCN_c10', 'VCN_c13']
VCN_c09 VCN_c02
Cell: 09: Soma mesh area: 1340.36  Soma hoc area: 885.30        Soma Inflation ratio: 1.514


TypeError: unsupported format string passed to dict.__format__

In [11]:
inps = ['self']*4 
cells = ['VCN_c09', 'VCN_c11', 'VCN_c17', 'VCN_c18']
print(inps)
for i, cell in enumerate(cells):
    print(cell, inps[i])
    fnsl = get_sims(cell, inps[i])
    plot_sim(cell, inps[i])

['self', 'self', 'self', 'self']
VCN_c09 self
Cell: 09: Soma mesh area: 1340.36  Soma hoc area: 885.30        Soma Inflation ratio: 1.514
fnb:  VCN_Cells/VCN_c09/Simulations/AN/AN_Result_VCN_c09_inp=self_XM13_nacncoop_II_soma=1.514_dend=1.236_ASA=1.514_Syn000_multisite_001_tonepip_030dB_16000.0_MS.p
name:  AN_Result_VCN_c09_inp=self_XM13_nacncoop_II_soma=1.514_dend=1.236_ASA=1.514_Syn000_multisite_001_tonepip_030dB_16000.0_MS.p
name:  AN_Result_VCN_c09_inp=self_XM13_nacncoop_II_soma=1.514_dend=1.236_ASA=1.514_Syn001_multisite_001_tonepip_030dB_16000.0_MS.p
name:  AN_Result_VCN_c09_inp=self_XM13_nacncoop_II_soma=1.514_dend=1.236_ASA=1.514_Syn002_multisite_001_tonepip_030dB_16000.0_MS.p
name:  AN_Result_VCN_c09_inp=self_XM13_nacncoop_II_soma=1.514_dend=1.236_ASA=1.514_Syn003_multisite_001_tonepip_030dB_16000.0_MS.p
name:  AN_Result_VCN_c09_inp=self_XM13_nacncoop_II_soma=1.514_dend=1.236_ASA=1.514_Syn004_multisite_001_tonepip_030dB_16000.0_MS.p
name:  AN_Result_VCN_c09_inp=self_XM13_nacnc